# FAISS를 활용법과 ODQA 완성하기
이전 미션 마지막 질문은 팀원들과 고민해보셨나요? 아무래도 동시에 많은 사용자가 Query 를 보내는 상황을 고려했을 때, GPU 를 사용하더라도 유사한 Passage 를 빠르게 찾아서 반환하기에는 시간이 조금 걸릴 것 같네요. 그 해답을 이번 6강에서 배운 Faiss 를 통해 해결해봅시다.

그러면 우리는 성공적으로 빠르고 정확하게 Passage Retrieval 을 할 수 있게 되었습니다. 하지만 QA 모델을 완성하려면, 찾아낸 Passage 에서 답을 찾아내는 과정도 필요하겠죠. Retrieval 된 Passage 로부터 답을 찾아내는 Reader 모델까지 연결시키는 코드를 마무리해봅시다.

```
🛠 Setup을 하는 부분입니다. 이전 미션에서 반복되는 부분이기 때문에 무지성 실행 하셔도 좋습니다.
💻 실습 코드입니다. 따라가면서 코드를 이해해보세요.
❓ 미션입니다. 주어진 질문과 요구사항에 맞춰서 직접 코드를 짜보세요.
```

Faiss 관련 추가 자료
+ https://www.pinecone.io/learn/faiss-tutorial/
+ [Difference between Voronoi & Kmeans](https://www.quora.com/What-is-the-difference-between-K-Means-and-Voronoi)

## 🛠 초기설정

### 🛠 Requirements

In [ ]:
!pip install tqdm==4.64.1 -q
!pip install datasets==2.7.0 -q
!pip install transformers==4.24.0 -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 🛠 난수 고정 및 버전 확인

In [ ]:
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from pprint import pprint

import faiss

import torch
from torch.utils.data import RandomSampler, DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)

In [ ]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [ ]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("device:[%s]."%(device))

PyTorch version:[2.0.0+cu118].
device:[cuda:0].


### 🛠 데이터셋 로딩
KorQuAD 의 train 데이터를 학습 데이터로 활용

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")
corpus = list(set([example["context"] for example in dataset["train"]]))
print(f"총 {len(corpus)}개의 지문이 있습니다.")

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

총 9606개의 지문이 있습니다.


저번 실습에서는 DPR 구현에 초점을 뒀기 때문에 소량의 데이터만 활용했습니다. 이번에는 실습에서는 Faiss 를 통해 대량의 Passage 들과 유사도를 구해야하므로, 전체 Validation 데이터를 활용합니다.

In [ ]:
search_corpus = list(set([example["context"] for example in dataset["validation"]]))
print(f"총 {len(search_corpus)}개의 지문이 있습니다.")

총 960개의 지문이 있습니다.


### 🛠 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT 를 encoder 로 사용하므로, KLUE에서 제공하는 `klue/bert-base` tokenizer 를 활용해봅시다. 다른 pretrained 모델을 사용하고 싶으시다면, `model_checkpoint` 를 바꿔보세요 !

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

### 🛠 Dense Passage Retrieval 코드 가져오기
Faiss 를 사용하려면 우선 Passage 들이 전부 임베딩 되어있어야겠죠? 이 과정은 저번 실습 코드에서 Dense Retriever 를 활용해봅시다. 여러분이 완성하신 코드가 있으면 직접 활용해보세요. 없다면 저희가 제공드린 코드를 활용하셔도 무관합니다.

In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

# Pre-train된 모델을 사용해줍니다. 위에서 사용한 `model_checkpoint`를 재활용합니다.
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.seq_relationsh

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(
    search_corpus,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
valid_dataset = TensorDataset(
    valid_p_seqs["input_ids"],
    valid_p_seqs["attention_mask"],
    valid_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

Iteration: 100%|██████████| 120/120 [00:32<00:00,  3.74it/s]


In [ ]:
p_embs = np.array(p_embs)
p_embs.shape # (num_passage, emb_dim)

(960, 768)

Question encoder를 활용해여 question dense embedding 생성

In [ ]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset["validation"])), 5)
query = dataset["validation"][sample_idx]["question"]
ground_truth = dataset["validation"][sample_idx]["context"]

query

['대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은?',
 '금강산의 겨울 이름은?',
 '유관순 열사는 당시 어떤 종교를 믿고 있었는가?',
 '1997년 10월 23일, 국회 본회의 대표 연설에서 전두환, 노태우 전 대통령에 대한 사면을 촉구한 새정치 국민회의 의원은?',
 '셰르징거가 찾아왔다가 우연히 푸시캣 돌스에 영입된 곳은?']

In [ ]:
valid_q_seqs = tokenizer(
    query,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    q_encoder.eval()
    q_embs = q_encoder(**valid_q_seqs).to("cpu").numpy()

torch.cuda.empty_cache()
q_embs.shape # (num_query, emb_dim)

(5, 768)

## 💻 방법에 따른 유사도를 구하는 데 걸리는 시간 측정하기
이제 검증셋에서 query 다섯 개를 뽑아왔으니 이제 이와 유사한 passage를 유사도를 통해 구해봅시다. 수업시간에 Faiss가 빠르다고 했는데 과연 정말로 빠를까요? 여러 가지 방법으로 유사도들을 구하는 데 걸리는 시간을 측정하고 비교해봅시다.

### 💻 GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [ ]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [ ]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

naive_gpu_time = time.time() - start_time
print(f"--- {naive_gpu_time:.6f} seconds ---")

tensor([[445, 902, 291,  ..., 851, 731, 725],
        [445,  78, 150,  ..., 851, 322, 725],
        [894, 150,  12,  ..., 239, 748, 329],
        [150, 894, 488,  ..., 731, 329, 348],
        [150, 894, 856,  ..., 725, 851,  52]], device='cuda:0')
--- 0.016702 seconds ---


In [ ]:
k = 5 

for i, q in enumerate(query[:1]):
    print("[Search query]\n", q, "\n")
    print("[Ground truth passage]")
    print(ground_truth[i], "\n")

    r = rank[i]
    for j in range(k):
        pprint("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
        pprint(search_corpus[r[j]])
    print("\n")

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

'Top-1 passage with score 111.7674'
("충주중학교를 졸업하였고, 충주고등학교 2학년 때 미국 적십자사에서 주최하는 영어경시대회에서 최고점수를 받았다. 부상으로 '외국 학생의 "
 "미국 방문 프로그램(VISTA)'에 선발되어 1962년 고등학교 3학년 때 미국을 방문했다. 한 달간 미국 연수 및 봉사활동에서 존 F. "
 '케네디 대통령을 만나 외교관의 꿈을 키웠다. 1963년 충주고등학교를 수석으로 졸업하고 서울대학교 외교학과에 진학했다. 1965년 '
 '4월부터 약 2년 6개월간 육군 병장으로 군 복무를 마쳤다. 1970년 2월 서울대학교 외교학과 졸업과 동시에 제3회 외무고시에 차석으로 '
 '합격해 그 해 3월 외무부에 들어갔다. 신입 외교관 연수를 마칠 때 수석을 차지했다. UN 국제 영어, 프랑스어, 독일어, 일본어 소통이 '
 '가능하다.')
'Top-

### 💻 FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [ ]:
import faiss

num_clusters = 8

In [ ]:
emb_dim = p_embs.shape[-1]

quantizer = faiss.IndexFlatL2(emb_dim)

indexer = faiss.IndexIVFScalarQuantizer(
    quantizer,
    quantizer.d,
    num_clusters,
    faiss.METRIC_L2
)
indexer.train(p_embs)
indexer.add(p_embs)

In [ ]:
# 3. Search using indexer
start_time = time.time()
D, I = indexer.search(q_embs, k)

faiss_cpu_time = time.time() - start_time
print(f"--- {faiss_cpu_time:.6f} seconds ---")

--- 0.003455 seconds ---


놀라운 결과네요. 몇 배나 차이나는지 비교해볼까요?

In [ ]:
print(f"FAISS가 GPU에 비해 {naive_gpu_time/faiss_cpu_time:.3f}배 빠르네요 !")

FAISS가 GPU에 비해 4.834배 빠르네요 !


Faiss 는 상위 k 개의 passage 에 대해서 각각의 거리(=유사도)와 해당 passage의 
인덱스를 아래와 같이 반환해줍니다.

In [ ]:
print("[Distance]")
print(D)
print("\n")
print("[Index of Top-5 Passages]")
print(I)

[Distance]
[[309.27466 333.93518 336.45502 338.7136  339.02985]
 [326.69012 347.01202 365.69153 367.72974 369.0745 ]
 [322.2229  363.2237  384.8753  387.86206 388.1231 ]
 [270.607   274.27563 305.33478 313.0304  313.21487]
 [308.95312 325.31995 355.09082 368.5417  382.0644 ]]


[Index of Top-5 Passages]
[[150 894 171 902 550]
 [150 894 550 171 198]
 [894 150 550 717  12]
 [150 894 717 621 488]
 [150 894 717 171 624]]


실제로 Passage 가 잘 뽑혔는지 확인해봅시다.   
혹시 결과가 잘 안나온다면 Encoder 가 학습이 제대로 되지 않아서 그렇습니다. Sparse Retrieval 이나 다른 방법을 시도해보세요.

In [ ]:
for i, q in enumerate(query[:1]):
    print("[Search query]\n", q, "\n")
    print("[Ground truth passage]")
    print(ground_truth[i], "\n")

    d = D[i]
    i = I[i]
    for j in range(k):
        pprint("Top-%d passage with distance %.4f" % (j+1, d[j]))
        pprint(search_corpus[i[j]])
    print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

'Top-1 passage with distance 309.2747'
('천사장(archangel)에 해당하는 영어 단어의 접두사(arch)는 “수석” 혹은 “우두머리”를 뜻하는 것으로 천사장 즉 수석 천사가 '
 '하나뿐임을 시사한다. 성경에서 “천사장”이 복수 형태로 나오는 경우는 결코 없다. 데살로니가 첫째 4:16에서는 천사장의 탁월함과 그 '
 '직무의 권위에 대해 말하면서 부활되신 주 예수 그리스도를 그런 식으로 부른다. “주께서 친히 호령과 천사장의 음성과 하느님의 나팔과 함께 '
 '하늘에서 내려오실 것이며, 그리스도와 결합하여 죽어 있는 사람들이 먼저 일어날 것[입니다].” 그러므로 “천사장”이란 단어와 직접 '
 '관련되어 있는 이름이 미가엘뿐이라는 사실은 의미 깊은 것이다.—유 9. 미가엘 1번 참조. (출처 : Insight on the '
 'Scriptures)')
'T

### ❓ Faiss 를 활용한 Retriever 를 class 로 작성해봅시다!

이렇게 빠른 Faiss 를 저렇게 흩날리는 코드로 사용하면 다 소용이 없겠죠? 저번 미션처럼 class로 구현해보면 훨씬 좋습니다! 이 class는 지난 번 학습한 Sparse Passage Retreival 혹은 Dense Passage Retrieval 방법으로 얻어낸 `p_embs` 를 인자로 받아 Faiss Indexer 를 Build 하고 `get_relevant_doc` 메소드를 통해 `query`를 입력하고 유사도가 높은 상위 `k`개의 passage 의 index 를 반환하는 코드를 작성해봅니다.   
여유가 되신다면 `faiss.Clustering` 을 활용할 때 사용되는 `num_clusters`, `niter` 도 인자로 받아봅시다.

In [ ]:
#오늘도 #즐거운 #미션

class FaissRetrieval:

    def __init__(self, p_embs):

        """
        Arguments:
            p_embs (torch.Tensor):
                위에서 사용한 Passage Encoder로 구한
                전체 Passage들의 Dense Representation을 받아옵니다.
                
        Summary:
            초기화하는 부분
            `build_faiss` 메소드도 여기서 수행하면 좋을 것 같습니다.
        """
        
        pass
        
    def build_faiss(self):

        """
        Note:
            위에서 Faiss를 사용했던 기억을 떠올려보면,
            Indexer를 구성해서 .search() 메소드를 활용했습니다.
            여기서는 Indexer 구성을 해주도록 합시다.
        """

        pass

    def get_relevant_doc(self, query, k=1):

        """
        Arguments:
            query (torch.Tensor):
                Dense Representation으로 표현된 query를 받습니다.
                문자열이 아님에 주의합시다.
            k (int, default=1):
                상위 몇 개의 유사한 passage를 뽑을 것인지 결정합니다.
        
        Note:
            받은 query를 이 객체에 저장된 indexer를 활용해서
            유사한 문서를 찾아봅시다.
            
            이전 미션의 Sparse Retrieval에서는 query로 문자열을 받아왔지만,
            이번에는 torch.Tensor를 받아오네요.
            현재 우리가 만들 FaissRetrieval는
            별도로 문자열을 Vector Representation으로 변환하는 역할은 하지 않기 때문입니다.
            SparseRetrieval의 경우, TF-IDF를 통해 직접 Sparse Representation으로 변환하는 과정을 거쳤지만,
            FaissRetrieval은 단순히 빌드된 indexer에서 관련 문서를 찾아주는 역할만 수행함에 유의해주세요.
        """
        
        pass

아래와 같이 간단하게 활용이 가능하다면 수고하셨습니다. 반드시 아래의 형태와 메소드명을 따라가지 않아도 좋습니다. 기호에 맞게 코드를 수정해보세요 !

In [ ]:
query = "금강산의 겨울 이름은?"
q_emb = ... # 임베딩된 Query Embedding을 가져옵시다 !

# p_embs는 처음에 만든 embedding을 이용합시다.
retriever = FaissRetrieval(p_embs)
retriever.build_faiss()
results = retriever.get_relevant_doc(q_emb, k=5)

In [ ]:
print("[Search query]\n", query, "\n")

print(f"Top-{len(results[0])} passages")
for d, i in zip(*results):
    print(f"Distance {d:.5f} | Passage {i}")
    pprint(search_corpus[i])
print("\n")

[Search query]
 금강산의 겨울 이름은? 

Top-5 passages
Distance 325.65179 | Passage 548
('천사장(archangel)에 해당하는 영어 단어의 접두사(arch)는 “수석” 혹은 “우두머리”를 뜻하는 것으로 천사장 즉 수석 천사가 '
 '하나뿐임을 시사한다. 성경에서 “천사장”이 복수 형태로 나오는 경우는 결코 없다. 데살로니가 첫째 4:16에서는 천사장의 탁월함과 그 '
 '직무의 권위에 대해 말하면서 부활되신 주 예수 그리스도를 그런 식으로 부른다. “주께서 친히 호령과 천사장의 음성과 하느님의 나팔과 함께 '
 '하늘에서 내려오실 것이며, 그리스도와 결합하여 죽어 있는 사람들이 먼저 일어날 것[입니다].” 그러므로 “천사장”이란 단어와 직접 '
 '관련되어 있는 이름이 미가엘뿐이라는 사실은 의미 깊은 것이다.—유 9. 미가엘 1번 참조. (출처 : Insight on the '
 'Scriptures)')
Distance 345.54416 | Passage 161
('1979년에 “중화인민공화국 형법”이 제정되기까지는 단행 법령이나 각종 사법해석, 공산당의 문서 등에 형벌규칙을 두고 있었다. 1979년 '
 '형법은 범죄를 “사회에 위해를 가하는 행위로서, 법률에 의한 형벌을 받을 수 있는 것”이라고 정의하여, 유추해석을 공인하고 있다. '
 '1997년에 형법이 전면적으로 개정되었다. 그 이후, 전인대 상무위원회에 의한 다수의 개정이 있다. 1997년 형법은 유추해석을 '
 '금지하여, 죄형법정주의를 채택하였다. 한국, 일본을 비롯한 대륙법권의 형법과 비교할 때 큰 특색으로서는, 공범론에 있어서, 정범 · '
 '종범이라고 하는 구성요건을 중심으로 한 구조 대신에, 주범 · 종범이라고 하는 범죄의 경위에 착안한 구조를 이용하고 있다. 주형에는 '
 '관제(공안기관의 감독하에 생활하는 것), 구역(노동개조형), 유기징역, 무기징역, 사형의 5종류가 있고, 부가형으로는 벌금, 정치적 '
 '권리박탈, 재산 

❗ Hint

Scratch 부터 짜는게 많이 어려우신가요?
+ 만들어야하는 기능들의 파이프라인을 나열하고 하나씩 함수화하는 것이 좋습니다. 

#### ➕추가 미션: SparseRetrieval Class로 합치기
3번 미션에서 TF-IDF 를 통해 SparseRetrieval 클래스를 짜셨나요? 여기에서 retrieve 라는 메소드를 통해 우리가 원하는 query 와 유사한 passage 를 구했는데 이번에 이 코드를 모두 Faiss 를 이용해 구해보도록 메소드를 추가해봅시다.

## ✔ 미션를 마무리하며 ...
미션의 난이도가 조금 있었는데 수행하시느라 고생 많으셨습니다. 대부분 미션에서 모듈화, 클래스화를 권장했는데요! 엔지니어로서 코드 디자인에 대한 감각을 익힐 뿐 아니라, 기계독해 대회의 베이스라인 이해도를 높이기 위해서였습니다. 실습과 미션를 성실히 수행한 후 베이스라인을 다시 보았을 때, 이해도가 올라가있는 여러분들을 발견하셨을 거라 믿습니다. 질문이 있으면 언제든 슬랙에 남겨주세요 ! 남은 대회기간, 부스트캠프 기간 모두 화이팅입니다!

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

